# Usage demo for NormalLikelihoodVariableSelector

In [1]:
import numpy as np
import pandas as pd
from millipede import NormalLikelihoodVariableSelector

## First we create a demo dataset with 3 causal and 97 spurious features

In [2]:
num_datapoints = 50
num_covariates = 100

# create covariates
X = np.random.RandomState(0).randn(num_datapoints * num_covariates)
X = X.reshape((num_datapoints, num_covariates))

# specify the true causal coefficients
true_coefficients = np.array([1.0, -0.5, 0.25] + [0.0] * 97)
print("true_coefficients:\n", true_coefficients)

true_coefficients:
 [ 1.   -0.5   0.25  0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.  ]


In [3]:
observation_noise = 0.3
# compute responses using the true linear model and add additional observation noise
Y = X @ true_coefficients + observation_noise * np.random.RandomState(1).randn(num_datapoints)

# put the covariates and responses into a single numpy array
YX = np.concatenate([Y[:, None], X], axis=-1)
print("X.shape: ", X.shape, "  Y.shape: ", Y.shape, "  YX.shape: ", YX.shape)

X.shape:  (50, 100)   Y.shape:  (50,)   YX.shape:  (50, 101)


## Then we package the data as a Pandas DataFrame, giving each covariate a  unique name

In [4]:
columns = ['Response', 'Causal1', 'Causal2', 'Causal3']
columns += ['Spurious{}'.format(k) for k in range(1, 98)]
dataframe = pd.DataFrame(YX, columns=columns)
dataframe.head(5)

,Response,Causal1,Causal2,Causal3,Spurious1,Spurious2,Spurious3,Spurious4,Spurious5,Spurious6,...,Spurious88,Spurious89,Spurious90,Spurious91,Spurious92,Spurious93,Spurious94,Spurious95,Spurious96,Spurious97
0,2.295962,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,...,-0.403177,1.222445,0.208275,0.976639,0.356366,0.706573,0.010500,1.785870,0.126912,0.401989
1,2.055882,1.883151,-1.347759,-1.270485,0.969397,-1.173123,1.943621,-0.413619,-0.747455,1.922942,...,-1.292857,0.267051,-0.039283,-1.168093,0.523277,-0.171546,0.771791,0.823504,2.163236,1.336528
2,-0.133029,-0.369182,-0.239379,1.099660,0.655264,0.640132,-1.616956,-0.024326,-0.738031,0.279925,...,-0.628088,-0.481027,2.303917,-1.060016,-0.135950,1.136891,0.097725,0.582954,-0.399449,0.370056
3,-2.487024,-1.306527,1.658131,-0.118164,-0.680178,0.666383,-0.460720,-1.334258,-1.346718,0.693773,...,0.567290,-0.222675,-0.353432,-1.616474,-0.291837,-0.761492,0.857924,1.141102,1.466579,0.852552
4,0.410583,-0.598654,-1.115897,0.766663,0.356293,-1.768538,0.355482,0.814520,0.058926,-0.185054,...,-1.029935,-0.349943,1.100284,1.298022,2.696224,-0.073925,-0.658553,-0.514234,-1.018042,-0.077855


## Next we create a VariableSelector object appropriate for our continuous-valued responses

In [5]:
selector = NormalLikelihoodVariableSelector(dataframe,  # pass in the data
                                            'Response', # indicate the column of responses
                                            S=1.0,        # specify the expected number of covariates included a priori
                                            prior="isotropic" # specify the prior over the coefficients
                                           )

## Finally we run the MCMC algorithm to compute posterior inclusion probabilities (PIPs) and other posterior quantities of interest

In [6]:
selector.run(T=2000, T_burnin=1000, verbosity='bar', seed=2)

  0%|          | 0/3000 [00:00<?, ?it/s]

## The results are available in the selector.summary DataFrame

- As expected only the 3 causal covariates have PIPs. 
- In addition the true coefficients are identified correctly (up to noise).
- Note that the intercept term does not have a corresponding PIP, since it is always included in the model by assumption.

In [7]:
selector.summary

,PIP,Coefficient,Coefficient StdDev,Conditional Coefficient,Conditional Coefficient StdDev
Causal1,1.000000,1.035825e+00,0.037104,1.035825,0.037104
Causal2,0.999996,-4.488024e-01,0.050521,-0.448804,0.050514
Causal3,0.820367,1.801171e-01,0.094278,0.220393,0.044715
Spurious1,0.000294,1.571512e-05,0.001182,0.045993,0.044455
Spurious2,0.000777,7.542234e-05,0.003124,0.088072,0.060386
...,...,...,...,...,...
Spurious94,0.000182,-2.232366e-07,0.000780,-0.001378,0.061282
Spurious95,0.000149,-1.033008e-06,0.000386,-0.011474,0.039073
Spurious96,0.000192,6.161886e-06,0.000781,0.053105,0.049396
Spurious97,0.000198,6.448959e-06,0.000655,0.050267,0.028628


For example the largest spurious PIP is given by:

In [8]:
selector.summary.PIP.values[3:-1].max()

0.0028088194868103058

Some additional stats about the MCMC run are available in `selector.stats`:

In [9]:
selector.stats

{'Weight quantiles': '5/10/20/50/90/95:  4.28e-303  7.15e-06  6.14e-01  1.71e+01  2.51e+01  2.51e+01',
 'Weight moments': 'mean/std/min/max:  1.29e+01  1.13e+01  4.28e-303  2.51e+01',
 'Elapsed MCMC time': '3.9 seconds',
 'Mean iteration time': '1.309 ms',
 'Number of retained samples': 2000,
 'Number of burn-in samples': 1000}

## Using per-covariate prior inclusion probabilities

If we have additional prior information about which covariates are more or less likely a priori, 
we can provide this information by setting the `S` argument to a `pandas.Series` of covariate-specific
prior inclusion probabilities.

In [10]:
# let's make the 3rd covariate *less likely* a priori
S = np.ones(num_covariates) / num_covariates
S[2] *= 1.0e-4
S = pd.Series(S, index=columns[1:])
selector = NormalLikelihoodVariableSelector(dataframe, 'Response',
                                            S=S, prior="isotropic")

In [11]:
selector.run(T=2000, T_burnin=1000, verbosity='bar', seed=2)

  0%|          | 0/3000 [00:00<?, ?it/s]

### As expected, the PIP of the 3rd covariate is now very small

In [12]:
selector.summary

,PIP,Coefficient,Coefficient StdDev,Conditional Coefficient,Conditional Coefficient StdDev
Causal1,1.000000,1.053331,0.043744,1.053331,0.043744
Causal2,0.999977,-0.446203,0.060017,-0.446214,0.059977
Causal3,0.000449,0.000098,0.005025,0.239238,0.065160
Spurious1,0.000202,0.000004,0.000503,0.044658,0.033109
Spurious2,0.000840,0.000143,0.004322,0.106120,0.051495
...,...,...,...,...,...
Spurious94,0.000216,0.000007,0.000701,0.033538,0.035759
Spurious95,0.000147,-0.000002,0.000755,-0.016992,0.062673
Spurious96,0.000189,-0.000003,0.000552,-0.018755,0.041474
Spurious97,0.000156,0.000001,0.000680,0.007148,0.053374
